# Stock NeurIPS2018 Part 1. Data

This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [1]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [7]:
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /private/var/folders/dv/ly5hp_ms3bd9753868lg6c5w0000gn/T/pip-req-build-eczjzcza
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /private/var/folders/dv/ly5hp_ms3bd9753868lg6c5w0000gn/T/pip-req-build-eczjzcza
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 0496601ee2824ca675beb5a220afd9109ac02ffa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /private/var/folders/dv/ly5hp_ms3bd9753868lg6c5w0000gn/T/pip-install-lbr4kwvx/elegantrl_63ad78a3665e4a4b8efe367335a39cab
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /private/var/folders/dv/ly5hp_ms3bd9753868lg6c5w0000gn/T/pip-install-lbr4kwvx/elegantrl_63ad78a3665e4a4b8efe367335a39cab
  

In [2]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [3]:
aapl_df_yf = yf.download(tickers = "aapl", start='2020-01-01', end='2020-01-31')

[*********************100%***********************]  1 of 1 completed


In [4]:
aapl_df_yf.head()

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2020-01-02 00:00:00+00:00,72.876114,75.087502,75.150002,73.797501,74.059998,135480400
2020-01-03 00:00:00+00:00,72.167610,74.357498,75.144997,74.125000,74.287498,146322800
2020-01-06 00:00:00+00:00,72.742676,74.949997,74.989998,73.187500,73.447502,118387200
2020-01-07 00:00:00+00:00,72.400551,74.597504,75.224998,74.370003,74.959999,108872000
2020-01-08 00:00:00+00:00,73.565201,75.797501,76.110001,74.290001,74.290001,132079200


### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [5]:
aapl_df_finrl = YahooDownloader(start_date='2020-01-01', 
                                end_date='2020-01-31',
                                ticker_list=['aapl']).fetch_data()

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (20, 8)


In [6]:
aapl_df_finrl.head()

,date,open,high,low,close,volume,tic,day
0,2020-01-02,72.876114,75.087502,75.150002,74.059998,135480400,aapl,3
1,2020-01-03,72.167610,74.357498,75.144997,74.287498,146322800,aapl,4
2,2020-01-06,72.742676,74.949997,74.989998,73.447502,118387200,aapl,0
3,2020-01-07,72.400551,74.597504,75.224998,74.959999,108872000,aapl,1
4,2020-01-08,73.565201,75.797501,76.110001,74.290001,132079200,aapl,2


## Data for the chosen tickers

In [7]:
config_tickers.DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [8]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2021-10-29'

In [9]:
df_raw = YahooDownloader(start_date=TRAIN_START_DATE,
                         end_date=TRADE_END_DATE,
                         ticker_list=config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (94301, 8)


In [10]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,2.737004,3.241071,3.251429,3.067143,746015200,AAPL,4
1,2009-01-02,41.816147,58.990002,59.080002,58.590000,6547900,AMGN,4
2,2009-01-02,15.014072,19.330000,19.520000,18.570000,10955700,AXP,4
3,2009-01-02,33.941105,45.250000,45.560001,42.799999,7010200,BA,4
4,2009-01-02,30.602568,46.910000,46.980000,44.910000,7117200,CAT,4


# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.

**Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.

**Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take MACD as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [11]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [12]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list=INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature=False)
processed = fe.preprocess_data(df_raw)

Successfully added technical indicators


[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (3228, 8)
Successfully added vix
Successfully added turbulence index


In [13]:
processed

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,2.737004,3.241071,3.251429,3.067143,746015200,AAPL,4,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,3.067143,3.067143,39.580002,0.000000
1,2009-01-02,41.816147,58.990002,59.080002,58.590000,6547900,AMGN,4,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,58.590000,58.590000,39.580002,0.000000
2,2009-01-02,15.014072,19.330000,19.520000,18.570000,10955700,AXP,4,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,18.570000,18.570000,39.580002,0.000000
3,2009-01-02,33.941105,45.250000,45.560001,42.799999,7010200,BA,4,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,42.799999,42.799999,39.580002,0.000000
4,2009-01-02,30.602568,46.910000,46.980000,44.910000,7117200,CAT,4,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,44.910000,44.910000,39.580002,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93607,2021-10-27,434.553558,453.480011,460.440002,454.640015,3520400,UNH,2,10.771091,460.483033,374.059963,64.578455,173.839875,52.124487,414.703666,416.023333,15.790000,65.838598
93608,2021-10-27,210.890610,215.779999,224.750000,224.750000,22958100,V,2,0.829003,236.085970,220.867026,46.873483,-118.134449,11.139468,227.208999,229.624999,15.790000,65.838598
93609,2021-10-27,43.580334,52.630001,53.200001,53.169998,15007400,VZ,2,-0.371777,55.281135,51.002865,45.194123,-45.795798,5.042716,53.521333,54.347334,15.790000,65.838598
93610,2021-10-27,39.633938,47.119999,48.459999,48.450001,5652000,WBA,2,0.136991,50.417853,45.842147,49.194606,-42.523628,9.578406,48.338667,48.670667,15.790000,65.838598


In [14]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed["date"].min(), processed["date"].max()).astype(str))
list_date

['2009-01-02',
 '2009-01-03',
 '2009-01-04',
 '2009-01-05',
 '2009-01-06',
 '2009-01-07',
 '2009-01-08',
 '2009-01-09',
 '2009-01-10',
 '2009-01-11',
 '2009-01-12',
 '2009-01-13',
 '2009-01-14',
 '2009-01-15',
 '2009-01-16',
 '2009-01-17',
 '2009-01-18',
 '2009-01-19',
 '2009-01-20',
 '2009-01-21',
 '2009-01-22',
 '2009-01-23',
 '2009-01-24',
 '2009-01-25',
 '2009-01-26',
 '2009-01-27',
 '2009-01-28',
 '2009-01-29',
 '2009-01-30',
 '2009-01-31',
 '2009-02-01',
 '2009-02-02',
 '2009-02-03',
 '2009-02-04',
 '2009-02-05',
 '2009-02-06',
 '2009-02-07',
 '2009-02-08',
 '2009-02-09',
 '2009-02-10',
 '2009-02-11',
 '2009-02-12',
 '2009-02-13',
 '2009-02-14',
 '2009-02-15',
 '2009-02-16',
 '2009-02-17',
 '2009-02-18',
 '2009-02-19',
 '2009-02-20',
 '2009-02-21',
 '2009-02-22',
 '2009-02-23',
 '2009-02-24',
 '2009-02-25',
 '2009-02-26',
 '2009-02-27',
 '2009-02-28',
 '2009-03-01',
 '2009-03-02',
 '2009-03-03',
 '2009-03-04',
 '2009-03-05',
 '2009-03-06',
 '2009-03-07',
 '2009-03-08',
 '2009-03-

In [29]:
combination = list(itertools.product(list_date,list_ticker))
combination

[('2009-01-02', 'AAPL'),
 ('2009-01-02', 'AMGN'),
 ('2009-01-02', 'AXP'),
 ('2009-01-02', 'BA'),
 ('2009-01-02', 'CAT'),
 ('2009-01-02', 'CRM'),
 ('2009-01-02', 'CSCO'),
 ('2009-01-02', 'CVX'),
 ('2009-01-02', 'DIS'),
 ('2009-01-02', 'GS'),
 ('2009-01-02', 'HD'),
 ('2009-01-02', 'HON'),
 ('2009-01-02', 'IBM'),
 ('2009-01-02', 'INTC'),
 ('2009-01-02', 'JNJ'),
 ('2009-01-02', 'JPM'),
 ('2009-01-02', 'KO'),
 ('2009-01-02', 'MCD'),
 ('2009-01-02', 'MMM'),
 ('2009-01-02', 'MRK'),
 ('2009-01-02', 'MSFT'),
 ('2009-01-02', 'NKE'),
 ('2009-01-02', 'PG'),
 ('2009-01-02', 'TRV'),
 ('2009-01-02', 'UNH'),
 ('2009-01-02', 'V'),
 ('2009-01-02', 'VZ'),
 ('2009-01-02', 'WBA'),
 ('2009-01-02', 'WMT'),
 ('2009-01-03', 'AAPL'),
 ('2009-01-03', 'AMGN'),
 ('2009-01-03', 'AXP'),
 ('2009-01-03', 'BA'),
 ('2009-01-03', 'CAT'),
 ('2009-01-03', 'CRM'),
 ('2009-01-03', 'CSCO'),
 ('2009-01-03', 'CVX'),
 ('2009-01-03', 'DIS'),
 ('2009-01-03', 'GS'),
 ('2009-01-03', 'HD'),
 ('2009-01-03', 'HON'),
 ('2009-01-03', 'IB

In [30]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,2.737004,3.241071,3.251429,3.067143,746015200.0,4.0,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,3.067143,3.067143,39.580002,0.000000
1,2009-01-02,AMGN,41.816147,58.990002,59.080002,58.590000,6547900.0,4.0,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,58.590000,58.590000,39.580002,0.000000
2,2009-01-02,AXP,15.014072,19.330000,19.520000,18.570000,10955700.0,4.0,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,18.570000,18.570000,39.580002,0.000000
3,2009-01-02,BA,33.941105,45.250000,45.560001,42.799999,7010200.0,4.0,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,42.799999,42.799999,39.580002,0.000000
4,2009-01-02,CAT,30.602568,46.910000,46.980000,44.910000,7117200.0,4.0,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,44.910000,44.910000,39.580002,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135773,2021-10-27,UNH,434.553558,453.480011,460.440002,454.640015,3520400.0,2.0,10.771091,460.483033,374.059963,64.578455,173.839875,52.124487,414.703666,416.023333,15.790000,65.838598
135774,2021-10-27,V,210.890610,215.779999,224.750000,224.750000,22958100.0,2.0,0.829003,236.085970,220.867026,46.873483,-118.134449,11.139468,227.208999,229.624999,15.790000,65.838598
135775,2021-10-27,VZ,43.580334,52.630001,53.200001,53.169998,15007400.0,2.0,-0.371777,55.281135,51.002865,45.194123,-45.795798,5.042716,53.521333,54.347334,15.790000,65.838598
135776,2021-10-27,WBA,39.633938,47.119999,48.459999,48.450001,5652000.0,2.0,0.136991,50.417853,45.842147,49.194606,-42.523628,9.578406,48.338667,48.670667,15.790000,65.838598


In [31]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])
processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,2.737004,3.241071,3.251429,3.067143,746015200.0,4.0,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,3.067143,3.067143,39.580002,0.000000
1,2009-01-02,AMGN,41.816147,58.990002,59.080002,58.590000,6547900.0,4.0,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,58.590000,58.590000,39.580002,0.000000
2,2009-01-02,AXP,15.014072,19.330000,19.520000,18.570000,10955700.0,4.0,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,18.570000,18.570000,39.580002,0.000000
3,2009-01-02,BA,33.941105,45.250000,45.560001,42.799999,7010200.0,4.0,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,42.799999,42.799999,39.580002,0.000000
4,2009-01-02,CAT,30.602568,46.910000,46.980000,44.910000,7117200.0,4.0,0.000000,3.565522,2.829121,100.000000,66.666667,100.000000,44.910000,44.910000,39.580002,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135773,2021-10-27,UNH,434.553558,453.480011,460.440002,454.640015,3520400.0,2.0,10.771091,460.483033,374.059963,64.578455,173.839875,52.124487,414.703666,416.023333,15.790000,65.838598
135774,2021-10-27,V,210.890610,215.779999,224.750000,224.750000,22958100.0,2.0,0.829003,236.085970,220.867026,46.873483,-118.134449,11.139468,227.208999,229.624999,15.790000,65.838598
135775,2021-10-27,VZ,43.580334,52.630001,53.200001,53.169998,15007400.0,2.0,-0.371777,55.281135,51.002865,45.194123,-45.795798,5.042716,53.521333,54.347334,15.790000,65.838598
135776,2021-10-27,WBA,39.633938,47.119999,48.459999,48.450001,5652000.0,2.0,0.136991,50.417853,45.842147,49.194606,-42.523628,9.578406,48.338667,48.670667,15.790000,65.838598


In [32]:
processed_full.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,2.737004,3.241071,3.251429,3.067143,746015200.0,4.0,0.0,3.565522,2.829121,100.0,66.666667,100.0,3.067143,3.067143,39.580002,0.0
1,2009-01-02,AMGN,41.816147,58.990002,59.080002,58.590000,6547900.0,4.0,0.0,3.565522,2.829121,100.0,66.666667,100.0,58.590000,58.590000,39.580002,0.0
2,2009-01-02,AXP,15.014072,19.330000,19.520000,18.570000,10955700.0,4.0,0.0,3.565522,2.829121,100.0,66.666667,100.0,18.570000,18.570000,39.580002,0.0
3,2009-01-02,BA,33.941105,45.250000,45.560001,42.799999,7010200.0,4.0,0.0,3.565522,2.829121,100.0,66.666667,100.0,42.799999,42.799999,39.580002,0.0
4,2009-01-02,CAT,30.602568,46.910000,46.980000,44.910000,7117200.0,4.0,0.0,3.565522,2.829121,100.0,66.666667,100.0,44.910000,44.910000,39.580002,0.0


# Part 4: Save the Data

### Split the data for training and trading

In [36]:
train = data_split(processed_full, TRAIN_START_DATE, TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE, TRADE_END_DATE)
print(len(train))
print(len(trade))

83897
9715


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.
For users running on your local environment, the csv files should be at the same directory of this notebook.

In [37]:
train.to_csv("train_data.csv")
trade.to_csv("trade_data.csv")